<a href="https://colab.research.google.com/github/Ronin1289/Generating-Building-Drawings-Using-Image-Processing/blob/main/Sample_project_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To be run locally on jupyter/vs code

In [ ]:
pip install svgwrite

In [ ]:
import cv2
import numpy as np

# Load the image
image_path = "gsck1.jpeg"  # Replace with your image path
image = cv2.imread(image_path)
clone = image.copy()  # Make a copy to draw points


In [ ]:

# Global variable to store selected points
src_points = []

def select_points(event, x, y, flags, param):
    """Mouse callback function to capture 4 points"""
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse click
        src_points.append((x, y))
        print(f"Selected point: {src_points[-1]}")

        # Draw a small circle on the image where the point was clicked
        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Select 4 Points", clone)

        if len(src_points) == 4:
            cv2.destroyAllWindows()  # Close window after selecting 4 points

# Open a window for selecting points
cv2.imshow("Select 4 Points", image)
cv2.setMouseCallback("Select 4 Points", select_points)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()  # Ensure the window closes

print("Selected source points:", src_points)

# Ensure exactly 4 points were selected
if len(src_points) != 4:
    print("Error: You must select exactly 4 points.")
    exit()


In [ ]:
# src_points=[(58, 391), (1218, 351), (59, 577), (1231, 582)]
# Define destination points (rectangle in output image)
dst_pts = [(0, 0), (600, 0), (0, 300), (600, 300)]
output_size = (600, 300)

def warp_perspective(image, src_points, dst_points, output_size):
    """Applies perspective warp to the image."""
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)

    # Compute the perspective transform matrix
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply warp
    warped_image = cv2.warpPerspective(image, matrix, output_size)

    return warped_image

# Apply perspective warp
warped = warp_perspective(image, src_points, dst_pts, output_size)

# Show the warped image
cv2.imshow("Warped Image", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite("warped_output.jpg",op)

In [ ]:
import cv2
import numpy as np
import svgwrite


In [ ]:

def image_to_svg(image):
    """
    Converts an image (NumPy array) to SVG format using edge detection and contour tracing.

    :param image: Input image as a NumPy array (BGR or grayscale).
    :return: SVG string data
    """
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection (fixed thresholds 70 and 100)
    edges = cv2.Canny(image, 70, 100)

    # Find contours from the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get image dimensions
    h, w = image.shape

    # Create an SVG drawing
    dwg = svgwrite.Drawing(size=(w, h))

    # Convert contours to SVG paths
    for contour in contours:
        points = [(p[0][0], p[0][1]) for p in contour]  # Convert to (x, y) tuples
        if len(points) > 2:  # Only add meaningful paths
            dwg.add(dwg.polyline(points, stroke="black", fill="none", stroke_width=1))

    return dwg.tostring()

# Example usage
# image = cv2.imread("input.jpg")  # Load an image
svg_data = image_to_svg(warped)  # Convert to SVG string

# Save the SVG to a file (optional)
with open("output.svg", "w") as f:
    f.write(svg_data)

print("SVG conversion complete.")


In [ ]:
def image_to_svg(image):
    """
    Converts an image (NumPy array) to SVG format using edge detection and contour tracing.

    :param image: Input image as a NumPy array (BGR or grayscale).
    :return: SVG string data
    """
    # Convert to grayscale if the image is in color
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection (fixed thresholds 70 and 100)
    edges = cv2.Canny(image, 70, 100)

    # Find contours from the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get image dimensions
    h, w = image.shape

    # Create an SVG drawing
    dwg = svgwrite.Drawing(size=(w, h))

    # Convert contours to SVG paths
    for contour in contours:
        points = [(float(p[0][0]), float(p[0][1])) for p in contour]  # Ensure coordinates are tuples
        if len(points) > 2:  # Only add meaningful paths
            dwg.add(dwg.polyline(points, stroke="black", fill="none", stroke_width=1))

    return dwg.tostring()

# Example usage
# image = cv2.imread("input.jpg")  # Load an image
svg_data = image_to_svg(warped)  # Convert to SVG

# Save the SVG to a file (optional)
with open("Approach 1.svg", "w") as f:
    f.write(svg_data)

print("SVG conversion complete.")


Gives output a svg file to be converted into dxf using inkscape
